In [ ]:
import jax

jax.config.update("jax_enable_x64", True)

import equinox as eqx
import jax.numpy as jnp
import matplotlib.pyplot as plt

from optimal_control.environments.examples import FibrosisEnvironment
from optimal_control.solvers import InterpolationControl


In [ ]:
control = jnp.sin(
    jnp.stack(
        (jnp.linspace(0.0, 10.0, 100), jnp.linspace(0.0, 5.0, 100) + 0.33), axis=-1
    )
)
t = jnp.linspace(-10.0, 120.0, 1000)

linear_control = InterpolationControl(2, 100, 0.0, 100.0, "step", control)

control_signal = linear_control(t)

plt.figure()
plt.plot(t, control_signal)
plt.show()


In [ ]:
environment = FibrosisEnvironment()
environment_state = environment.init()

print(environment_state)

control = InterpolationControl(2, 100, 0.0, 100.0, "linear", jnp.ones((100, 2)))
sequence = environment.integrate(control, environment_state)

print(sequence)
